In [31]:
from ultralytics import YOLO
import os
import pytesseract
import cv2
import numpy

In [32]:
# Load the model
model = YOLO("/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/Server/model/versions/0.2/receipt_extractor.pt")

In [33]:
image_path = "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/receipt extraction model.v2i.yolov8/test/images/0676aa86-IMG_20231221_163400_jpg.rf.3ef3b7a564323dc7d96c82e257c4c400.jpg"
annotation_path = "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/receipt extraction model.v2i.yolov8/test/labels/0676aa86-IMG_20231221_163400_jpg.rf.3ef3b7a564323dc7d96c82e257c4c400.txt"

In [34]:
def get_highest_prediction(model, image_path)-> dict:
    # Get the predictions
    predictions = model(image_path)

    # Get the highest prediction
    highest_conf_detections = {}
    for result in predictions:
        # print(result)
        for box in result.boxes:
            class_id = int(box.cls)
            class_label = result.names[int(box.cls)]
            conf = box.conf

            # Check if the class label is already in the dictionary
            if class_label in highest_conf_detections:
                # Check if the current confidence is higher than the one stored
                if conf > highest_conf_detections[class_label]["conf"]:
                    highest_conf_detections[class_label] = {
                    "conf": conf,
                    "class_label": class_label,
                    "class_id": class_id,
                    "coordinates": {
                        "x1": int(box.xyxy[0][0]),
                        "y1": int(box.xyxy[0][1]),
                        "x2": int(box.xyxy[0][2]),
                        "y2": int(box.xyxy[0][3]),
                    }
                }
            else:
                highest_conf_detections[class_label] = {
                    "conf": conf,
                    "class_label": class_label,
                    "class_id": class_id,
                    "coordinates": {
                        "x1": int(box.xyxy[0][0]),
                        "y1": int(box.xyxy[0][1]),
                        "x2": int(box.xyxy[0][2]),
                        "y2": int(box.xyxy[0][3]),
                    }
                }


    return highest_conf_detections

extracted_prediction = get_highest_prediction(model, image_path)
print(extracted_prediction)


image 1/1 /Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/receipt extraction model.v2i.yolov8/test/images/0676aa86-IMG_20231221_163400_jpg.rf.3ef3b7a564323dc7d96c82e257c4c400.jpg: 640x640 1 date, 1 item_purchase, 1 shop_information, 1 total, 538.1ms
Speed: 0.9ms preprocess, 538.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
{'shop_information': {'conf': tensor([0.8474]), 'class_label': 'shop_information', 'class_id': 2, 'coordinates': {'x1': 194, 'y1': 129, 'x2': 462, 'y2': 209}}, 'date': {'conf': tensor([0.7863]), 'class_label': 'date', 'class_id': 0, 'coordinates': {'x1': 150, 'y1': 212, 'x2': 314, 'y2': 234}}, 'total': {'conf': tensor([0.7854]), 'class_label': 'total', 'class_id': 3, 'coordinates': {'x1': 140, 'y1': 453, 'x2': 472, 'y2': 477}}, 'item_purchase': {'conf': tensor([0.6117]), 'class_label': 'item_purchase', 'class_id': 1, 'coordinates

In [17]:

def convert_yolo_coordinates(file_path, img_width, img_height)->dict:

    class_mapping = {
        0: "date",
        1: "item_purchase",
        2: "shop_information",
        3: "total",
    }
        
    labels_coordinates = {}

    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue  # Skip invalid lines

        class_id, x_center, y_center, width, height = map(float, parts)

        # Calculate the coordinates of the bounding box (x1, y1, x2, y2)
        x1 = int((x_center - width / 2) * img_width)
        y1 = int((y_center - height / 2) * img_height)
        x2 = int((x_center + width / 2) * img_width)
        y2 = int((y_center + height / 2) * img_height)

        labels_coordinates[class_mapping[int(class_id)]] = {
            "x1": x1,
            "y1": y1,
            "x2": x2,
            "y2": y2
        }
    return labels_coordinates

convert_yolo_coordinates(annotation_path, 640, 640)


{'shop_information': {'x1': 129, 'y1': 125, 'x2': 414, 'y2': 179},
 'item_purchase': {'x1': 72, 'y1': 179, 'x2': 449, 'y2': 367},
 'total': {'x1': 96, 'y1': 366, 'x2': 420, 'y2': 382},
 'date': {'x1': 59, 'y1': 545, 'x2': 244, 'y2': 565}}

In [18]:
def calculate_accuracy(model, image_path, annotation_path):
    predictions = get_highest_prediction(model, image_path)
    labels = convert_yolo_coordinates(annotation_path, 640, 640)
    accuracies = {}

    # Initialize all classes with None
    for class_name in labels.keys():
        accuracies[class_name] = None

    #for each prediction, display the bounding box on the image and save it
    print(f"label")
    for key in labels:

        if key not in predictions:
            print(f"{key} not found")
            accuracies[key] = None
            continue

        #load the image
        image = cv2.imread(image_path)

        # print(f"{key} : {predictions[key]['class_id']}")

        real_coordinates = labels[key]
        predicted_coordinates = predictions[key]["coordinates"]
        
        # Draw the predicted bounding box with name
        x1 = predicted_coordinates["x1"]
        y1 = predicted_coordinates["y1"]
        x2 = predicted_coordinates["x2"]
        y2 = predicted_coordinates["y2"]
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, "prediction", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Draw the real bounding box with name
        x1 = real_coordinates["x1"]
        y1 = real_coordinates["y1"]
        x2 = real_coordinates["x2"]
        y2 = real_coordinates["y2"]
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(image, "real", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        # Calculate the common area between the two bounding boxes
        x1 = max(real_coordinates["x1"], predicted_coordinates["x1"])
        y1 = max(real_coordinates["y1"], predicted_coordinates["y1"])
        x2 = min(real_coordinates["x2"], predicted_coordinates["x2"])
        y2 = min(real_coordinates["y2"], predicted_coordinates["y2"])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 255), 2)
        cv2.putText(image, "intersection", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

        #save picture
        cv2.imwrite(f"/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/quick_test/prediction/{key}_prediction.png", image)

        #calculate the area of the union over the area of the prediction
        if x2 > x1 and y2 > y1:  # Ensure there is an intersection
            intersection_area = (x2 - x1) * (y2 - y1)
            union_area = (real_coordinates["x2"] - real_coordinates["x1"]) * (real_coordinates["y2"] - real_coordinates["y1"]) + \
                         (predicted_coordinates["x2"] - predicted_coordinates["x1"]) * (predicted_coordinates["y2"] - predicted_coordinates["y1"]) - \
                         intersection_area
            accuracy = intersection_area / union_area
            accuracies[key] = accuracy
            print(f"{key} accuracy: {accuracy}")
        else:
            accuracies[key] = None
            print(f"{key} has no intersection")

    return accuracies    
    
calculate_accuracy(model, image_path, annotation_path)


image 1/1 /Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/combine_annotated_data/test/images/IMG_20240329_145238_jpg.rf.b940f962c22ad6c46a413a05dd23b9c2.jpg: 640x640 2 item_purchases, 1 shop_information, 514.7ms
Speed: 1.1ms preprocess, 514.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
label
shop_information accuracy: 0.6092026872860946
item_purchase accuracy: 0.8045693700526141
total not found
date not found


{'shop_information': 0.6092026872860946,
 'item_purchase': 0.8045693700526141,
 'total': None,
 'date': None}

In [38]:
image_folder_path = "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/combine_annotated_data/valid/images/"
annotation_folder_path = "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/combine_annotated_data/valid/labels/"

model_paths = {
#    0.1 : "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/Server/model/versions/0.15/receipt_extractor.pt",
   0.2 : "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/Server/model/versions/0.2/receipt_extractor.pt",
}

for version, model_path in model_paths.items():
    model = YOLO(model_path)
    #list all the images in the folder wihout the extension
    images = [os.path.splitext(file)[0] for file in os.listdir(image_folder_path) if file.endswith(".jpg")]
    all_accuracy = {}
    for image in images:
        image_path = os.path.join(image_folder_path, image + ".jpg")
        annotation_path = os.path.join(annotation_folder_path, image + ".txt")
        accuracies = calculate_accuracy(model, image_path, annotation_path)
        all_accuracy[image] = accuracies

    all_accuracy

    # for each class calculate the average accuracy and the number of None
    average_accuracy = {}
    for key in all_accuracy[images[0]]:
        average_accuracy[key] = {
            "average_accuracy": 0,
            "number_of_none": 0
        }
        for image in images:
            if all_accuracy[image][key] is not None:
                average_accuracy[key]["average_accuracy"] += all_accuracy[image][key]
            else:
                average_accuracy[key]["number_of_none"] += 1
        average_accuracy[key]["average_accuracy"] = average_accuracy[key]["average_accuracy"] / (len(images) - average_accuracy[key]["number_of_none"])

    print("\n########################################################################\nAccuracy for version", version, ":")
    print(average_accuracy)

    overall_accuracy = 0
    for key in average_accuracy:
        overall_accuracy += average_accuracy[key]["average_accuracy"]
    overall_accuracy = overall_accuracy / len(average_accuracy)
    print(f"Overall accuracy: {overall_accuracy}")



image 1/1 /Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/combine_annotated_data/valid/images/c5ea012e-IMG_20240111_204434_jpg.rf.893dda8a9aa1d4c2f0d62be6eb3343a9.jpg: 640x640 1 date, 3 item_purchases, 1 shop_information, 1 total, 515.5ms
Speed: 0.8ms preprocess, 515.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
label
shop_information accuracy: 0.8898568761667703
item_purchase accuracy: 0.5757235643710096
date accuracy: 0.7181467181467182
total accuracy: 0.74051776038531

image 1/1 /Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/combine_annotated_data/valid/images/IMG_20240401_201118_jpg.rf.c948b0c3c0381619643571acc293a76d.jpg: 640x640 1 date, 2 item_purchases, 1 shop_information, 1 total, 506.0ms
Speed: 0.8ms preprocess, 506.0ms infe